In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import KFold

In [2]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment1/q6_7.xlsx')
df.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [3]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,18.27,16.09,0.8870,6.173,3.651,2.443,6.197,2
1,11.41,12.95,0.8560,5.090,2.775,4.957,4.825,3
2,13.50,13.85,0.8852,5.351,3.158,2.249,5.176,1
3,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
4,18.72,16.19,0.8977,6.006,3.857,5.324,5.879,2


In [4]:
datan = df.values
X = datan[:, 0:7]
X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
y = datan[:, 7]

In [5]:
num_classes = len(np.unique(y))
print(num_classes)

3


In [6]:
def MAP(X_inst, X, y):
  prior_1 = 0
  prior_2 = 0
  prior_3 = 0
  m = len(y)
  n = X.shape[1]
  for i in y:
    if (i==1):
      prior_1 = prior_1 + 1
    if (i==2):
      prior_2 = prior_2 + 1
    if (i==3):
      prior_3 = prior_3 + 1

  prior_1 = prior_1/m
  prior_2 = prior_2/m
  prior_3 = prior_3/m

  X1 = np.array([X[i] for (i, label) in enumerate(y) if label == 1])
  X2 = np.array([X[i] for (i, label) in enumerate(y) if label == 2])
  X3 = np.array([X[i] for (i, label) in enumerate(y) if label == 3])

  evidence1, evidence2, evidence3 = len(X1)/len(X), len(X2)/len(X), len(X3)/len(X)

  u1 = np.mean(X1, axis=0)
  u2 = np.mean(X2, axis=0)
  u3 = np.mean(X3, axis=0)

  covariance_matrix1 = np.cov(X1.T.astype(float))
  covariance_matrix2 = np.cov(X2.T.astype(float))                                       
  covariance_matrix3 = np.cov(X3.T.astype(float))

  coeffecient1 = 1/(((2*math.pi)**(n/2))*np.linalg.det(covariance_matrix1)**0.5)
  coeffecient2 = 1/(((2*math.pi)**(n/2))*np.linalg.det(covariance_matrix2)**0.5)     
  coeffecient3 = 1/(((2*math.pi)**(n/2))*np.linalg.det(covariance_matrix3)**0.5) 

  cov1_inv = np.linalg.inv(covariance_matrix1)
  cov2_inv = np.linalg.inv(covariance_matrix2)
  cov3_inv = np.linalg.inv(covariance_matrix3)
  likelihood1 = coeffecient1*np.exp(-0.5*np.dot(np.dot(X_inst-u1, cov1_inv), (X_inst-u1).T))
  likelihood2 = coeffecient2*np.exp(-0.5*np.dot(np.dot(X_inst-u2, cov2_inv), (X_inst-u2).T)) 
  likelihood3 = coeffecient3*np.exp(-0.5*np.dot(np.dot(X_inst-u3, cov3_inv), (X_inst-u3).T)) 

  prob1, prob2, prob3 = (likelihood1*prior_1)/(evidence1), (likelihood2*prior_2)/(evidence2), (likelihood3*prior_3)/(evidence3)
  if(max(prob1, prob2, prob3) == prob1):
    prediction = 1
  elif(max(prob1, prob2, prob3) == prob2):
    prediction = 2
  else:
    prediction = 3

  return prediction

In [7]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = len(labels)
  for i in range(m):
    conf_matrix[3*(labels[i].astype(int)-1) + predictions[i] - 1] = conf_matrix[3*(labels[i].astype(int)-1) + predictions[i] - 1] + 1 
  return conf_matrix

In [8]:
kf = KFold(n_splits=5)

ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall = []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  predictions = []
  for i in range(len(X_test)):
    predictions.append(MAP(X_test[i], X_train, y_train))
  conf_matrix = confusion_matrix(predictions,y_test)
  ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
  ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
  ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
  overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))

final_ind1 = sum(ind_acc1)/len(ind_acc1)
final_ind2 = sum(ind_acc2)/len(ind_acc2)
final_ind3 = sum(ind_acc3)/len(ind_acc3)
final_overall = sum(overall)/len(overall)

In [9]:
print("Mean Overall Accuracy: ", final_overall)
print("Mean Individual Accuracy for Class 1: ", final_ind1)
print("Mean Individual Accuracy for Class 2: ", final_ind2)
print("Mean Individual Accuracy for Class 3: ", final_ind3)

Mean Overall Accuracy:  0.9476190476190476
Mean Individual Accuracy for Class 1:  0.9179487179487179
Mean Individual Accuracy for Class 2:  0.9761403508771929
Mean Individual Accuracy for Class 3:  0.9564705882352941


In [10]:
print("Max Overall Accuracy: ", max(overall))
print("Max Individual Accuracy for Class 1: ", max(ind_acc1))
print("Max Individual Accuracy for Class 2: ", max(ind_acc2))
print("Max Individual Accuracy for Class 3: ", max(ind_acc3))

Max Overall Accuracy:  1.0
Max Individual Accuracy for Class 1:  1.0
Max Individual Accuracy for Class 2:  1.0
Max Individual Accuracy for Class 3:  1.0
